# Задача 1

Нужно написать две программы: Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian). Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии:
1. Простое последовательное чтение 
2. Многопоточная + memory-mapped files. Сравните время работы.

## Импорт библиотек

In [1]:
import numpy as np
import time
import mmap
import os
import threading
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor, Future
import random

## Вспомогательные функции

In [6]:
def timed_func(func):
    start_time = time.time()
    func()
    print(time.time() - start_time, "seconds")
    print("---------------------------------------")\


count = 70000000
file_name = 'big_endians.txt'

## Простая реализация

In [7]:
def simple_count():
    rng = random.SystemRandom(0)
    
    # file generation
    with open(file_name, "w") as file:
        for step in range(count):
            file.write(str(rng.randint(10**31, 10**32)) + '\n')
    
    # simple count
    sum_num, max_num, min_num = 0, 0, 10**32
    mas = []

    file = open(file_name)
    for line in file.readlines():
        num = int(line)
        if (num > max_num):
            max_num = num
        if (num < min_num):
            min_num = num
        sum_num += num
    file.close()

    print(sum_num, max_num, min_num)
    
timed_func(simple_count)
os.remove(file_name)

3850122694452320076938218838198386430459 99999993606373558972276846292936 10000000663292346932245350092448
181.20064496994019 seconds
---------------------------------------


## Многопоточная реализация

In [8]:
rng = random.SystemRandom(0)

sum_num = 0
max_num = 0
min_num = 10**32

lock_min = threading.Lock()
lock_max = threading.Lock()

dig = 32

def search(array, size):
        
    global sum_num
    global max_num
    global min_num
    
    for i in range(0, size):
        num = int(array[i*dig: (i+1)*dig])
        sum_num += num
        
        if num > max_num:
            max_num = num
        if num < min_num:
            min_num = num
    

def multithreads_count():
    try:   
        # file generation
        with open(file_name, 'w', encoding='utf-8') as f:
            for num in range(1, count):
                f.write(str(rng.randint(10**31, 10**32)))
        
        
        # multithreads count
        with open(file_name, 'r+b') as f:
            with mmap.mmap(f.fileno(), length=dig * (count-1), offset=0, access=mmap.ACCESS_WRITE) as mm:
                # print(mm[0:32])
                
                chunk = count // 4
                executor = ThreadPoolExecutor(max_workers=4)
                future1 = executor.submit(search, mm[0: chunk*dig], chunk) 
                future2 = executor.submit(search, mm[chunk*dig: 2*chunk*dig], chunk)
                future3 = executor.submit(search, mm[2*chunk*dig: 3*chunk*dig], chunk)
                future4 = executor.submit(search, mm[3*chunk*dig:], chunk)
    finally:
        os.remove(file_name)
        print(sum_num, max_num, min_num)

timed_func(multithreads_count)

95143381249640068573708990350763180210 99999952378697718053689289691467 10000007228893939576148706090991
113.73773074150085 seconds
---------------------------------------
